## 2. Cluster_PreparaDatos_DemandaMensual

### Objetivo

Proceso general de preparación de datos en donde se identificarán las variables que participarán en los procesos posteriores de clusterización. Se hace énfasis en la variable DEMANDA de viajes consolidado se abrirá para cada uno de los meses de funcionamiento.

### Descripción General de notebook

    1. Carga de datos base:
        - Movimientos históricos
        - Demografía por barrio
        - Geografía por estación Bicimad
        
    2. Concatenación de datasets
    3. Exportar DataFrame a CSV

## 1. Carga de datos base

In [2]:
import time
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

%run "../1. Librerias Mongo/MongoDB_Connections.ipynb"
%run "../1. Librerias Mongo/MongoDB_Funciones_Consultas.ipynb"

### Movimientos históricos

Carga de 2.912.138 registros históricos de viajes Bicimad

In [3]:
t_ini = time.time()
data_Tracks = pd.read_csv('../../Data/DataFrame_Final_Cierre_2017_2019.csv', parse_dates=['FECHA'])
t_end = time.time()
print (t_end - t_ini)

data_Tracks.shape

2.855964422225952


(2912138, 21)

In [4]:
data_Tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2912138 entries, 0 to 2912137
Data columns (total 21 columns):
 #   Column               Dtype         
---  ------               -----         
 0   ESTACION             int64         
 1   ANIO                 int64         
 2   MES                  int64         
 3   DIA                  int64         
 4   HORA                 int64         
 5   FECHA                datetime64[ns]
 6   DIA_SEMANA           int64         
 7   AM_PM                object        
 8   TEMPORADA            object        
 9   TEMPORADA_NUM        int64         
 10  Es_Festivo           int64         
 11  Es_FinSemana         int64         
 12  TEMPERATURA          float64       
 13  VIENTO               float64       
 14  PRESION              float64       
 15  HUMEDAD              float64       
 16  PRECIPITACION_1h     float64       
 17  PRECIPITACION_3h     float64       
 18  DESC_TIEMPO          object        
 19  DESC_TIEMPO_detalle  

In [5]:
data_Tracks.groupby('ESTACION').agg(DEMANDA=('DEMANDA', 'sum'), CUENTA=('DEMANDA', 'count'))

,DEMANDA,CUENTA
ESTACION,,
1,90192,19681
2,42031,14344
3,69541,19659
4,48318,16108
5,44461,17600
...,...,...
171,58379,17302
172,57917,17496
173,19783,10912


In [6]:
data_Tracks.groupby('ANIO').agg(DEMANDA=('DEMANDA', 'sum'))

,DEMANDA
ANIO,
2017,2741661
2018,3387010
2019,3645679
2020,476381


### Datos Demográficos por Barrio de Madrid

Los datos demográficos se encuentran cargados dentro de la colección "Demografia" en la base de datos BiciMAD en MongoDB Atlas

In [7]:
# DEMOGRAFIA
db_DemoG = _connect_mongo('cloud', 'cluster0.15npsxw.mongodb.net', None, 'ucmtfm2022', 'UCM_2022', 'BiciMAD', 'Demografia')
data_Demografia = _consulta_Demografia(db_DemoG)
data_Demografia

Conexion OK
Collection(Database(MongoClient(host=['ac-x1d17w2-shard-00-00.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-01.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-02.15npsxw.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-xyv6ql-shard-0', tls=True, serverselectiontimeoutms=4000, tlscafile='C:\\ProgramData\\Anaconda3\\lib\\site-packages\\certifi\\cacert.pem'), 'BiciMAD'), 'Demografia')


,Distrito,Distrito_Nombre,Barrio,Barrio_Nombre,Tasa_Paro,Renta_Media_Persona,Renta_Media_Hogar,Poblacion
0,1,Centro,01-03,Cortes,"4,087029691","20947,0742","40962,5241",10760
1,1,Centro,01-01,Palacio,"5,675629192","19443,00705","38973,53727",23708
2,1,Centro,01-05,Universidad,"4,283483014","17882,22351","35344,22565",33434
3,3,Retiro,03-01,Pacifico,"4,228419906","19527,32758","44831,75729",33879
4,3,Retiro,03-03,Estrella,"3,593663803","23696,69349","63495,10119",23504
...,...,...,...,...,...,...,...,...
126,19,Vicalvaro,19-02,Valdebernardo,"7,433348396","14438,77896","41477,93538",17851
127,20,San Blas - Canillejas,20-01,Simancas,"5,999410377","13298,0422","33592,67455",28799
128,20,San Blas - Canillejas,20-08,El Salvador,"4,93547958","21719,06345","57319,67856",11516
129,21,Barajas,21-02,Aeropuerto,"5,705474171",10330,27316,1911


In [8]:
data_Demografia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Distrito             131 non-null    int64 
 1   Distrito_Nombre      131 non-null    object
 2   Barrio               131 non-null    object
 3   Barrio_Nombre        131 non-null    object
 4   Tasa_Paro            131 non-null    object
 5   Renta_Media_Persona  131 non-null    object
 6   Renta_Media_Hogar    131 non-null    object
 7   Poblacion            131 non-null    int64 
dtypes: int64(2), object(6)
memory usage: 8.3+ KB


### Datos geográficos por Estación BiciMad

Los datos de ubicación geográfica de cada estación de BiciMad se encuentran cargados dentro de la colección "Estaciones" en la base de datos BiciMAD en MongoDB Atlas

In [9]:
db_Stations = _connect_mongo('cloud', 'cluster0.15npsxw.mongodb.net', None, 'ucmtfm2022', 'UCM_2022', 'BiciMAD', 'Estaciones')
data_Stations = _consulta_stations(db_Stations)

data_Stations = data_Stations.drop(columns='Finca')

data_Stations

Conexion OK
Collection(Database(MongoClient(host=['ac-x1d17w2-shard-00-00.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-01.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-02.15npsxw.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-xyv6ql-shard-0', tls=True, serverselectiontimeoutms=4000, tlscafile='C:\\ProgramData\\Anaconda3\\lib\\site-packages\\certifi\\cacert.pem'), 'BiciMAD'), 'Estaciones')


,Id_Estacion,Nro_Estacion,Gis_X,Gis_Y,Fec_Alta,Distrito,Distrito_Nombre,Barrio,Barrio_Nombre,Calle,Tipo_Reserva,Plazas,Longitud,Latitud,Direccion
0,6,5,"440447,06","4475539,6",41813,1,CENTRO,01-04,JUSTICIA,"FUENCARRAL, CALLE, DE",BiciMAD,27,-3.702074,40.428362,"FUENCARRAL, CALLE, DE, 106"
1,5,4,"440396,4","4475565,36",41813,1,CENTRO,01-05,UNIVERSIDAD,"MANUELA MALASA�A, CALLE, DE",BiciMAD,24,-3.702674,40.428590,"MANUELA MALASA�A, CALLE, DE, 3"
2,10,9,"439813,51","4474129,47",41813,1,CENTRO,01-01,PALACIO,"SAN MIGUEL, PLAZA, DE",BiciMAD,24,-3.709409,40.415613,"SAN MIGUEL, PLAZA, DE, 9"
3,12,11,"440314,86","4475395,19",41813,1,CENTRO,01-05,UNIVERSIDAD,"SAN ANDRES, CALLE, DE",BiciMAD,24,-3.703619,40.427052,"SAN ANDRES, CALLE, DE, 20"
4,13,12,"440095,53","4475560,73",41813,1,CENTRO,01-05,UNIVERSIDAD,"SAN BERNARDO, CALLE, DE",BiciMAD,24,-3.706220,40.428527,"SAN BERNARDO, CALLE, DE, 87"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,257,249,"441914,27","4481195,42",44194,5,CHAMARTiN,05-06,CASTILLA,"CASTELLANA, PASEO, DE LA",BiciMAD,24,-3.685296,40.479416,"CASTELLANA, PASEO, DE LA, frente al 298"
262,258,250,"441812,82","4476911,24",44194,5,CHAMARTiN,05-01,EL VISO,"SERRANO, CALLE, DE",BiciMAD,24,-3.686100,40.440815,"SERRANO, CALLE, DE, 113 b"
263,259,251,"443447,95","4477272,58",44194,5,CHAMARTiN,05-02,PROSPERIDAD,"SANTA HORTENSIA, CALLE, DE",BiciMAD,24,-3.666853,40.444183,"SANTA HORTENSIA, CALLE, DE, 31"
264,263,255,"441910,45","4479454,04",44194,5,CHAMARTiN,05-05,NUEVA ESPA�A,"GENERAL LOPEZ POZAS, CALLE, DEL",BiciMAD,24,-3.685182,40.463729,"GENERAL LOPEZ POZAS, CALLE, DEL, 2"


## 2. Concatenación (Join) de datasets

### Estaciones - Demografia

In [10]:
df_EstacionDemog = pd.merge(data_Stations, data_Demografia, how='left', 
                            left_on=['Distrito', 'Barrio'], 
                            right_on=['Distrito', 'Barrio'])

df_EstacionDemog = df_EstacionDemog.drop(columns=['Distrito_Nombre_y', 'Barrio_Nombre_y'])
df_EstacionDemog.rename(columns = {'Id_Estacion':'ESTACION', 'Distrito_Nombre_x':'Distrito_Nombre', 
                                   'Barrio_Nombre_x':'Barrio_Nombre'}, inplace = True)

In [11]:
df_EstacionDemog

,ESTACION,Nro_Estacion,Gis_X,Gis_Y,Fec_Alta,Distrito,Distrito_Nombre,Barrio,Barrio_Nombre,Calle,Tipo_Reserva,Plazas,Longitud,Latitud,Direccion,Tasa_Paro,Renta_Media_Persona,Renta_Media_Hogar,Poblacion
0,6,5,"440447,06","4475539,6",41813,1,CENTRO,01-04,JUSTICIA,"FUENCARRAL, CALLE, DE",BiciMAD,27,-3.702074,40.428362,"FUENCARRAL, CALLE, DE, 106","4,374558304","23759,66607","48202,79271",18072.0
1,5,4,"440396,4","4475565,36",41813,1,CENTRO,01-05,UNIVERSIDAD,"MANUELA MALASA�A, CALLE, DE",BiciMAD,24,-3.702674,40.428590,"MANUELA MALASA�A, CALLE, DE, 3","4,283483014","17882,22351","35344,22565",33434.0
2,10,9,"439813,51","4474129,47",41813,1,CENTRO,01-01,PALACIO,"SAN MIGUEL, PLAZA, DE",BiciMAD,24,-3.709409,40.415613,"SAN MIGUEL, PLAZA, DE, 9","5,675629192","19443,00705","38973,53727",23708.0
3,12,11,"440314,86","4475395,19",41813,1,CENTRO,01-05,UNIVERSIDAD,"SAN ANDRES, CALLE, DE",BiciMAD,24,-3.703619,40.427052,"SAN ANDRES, CALLE, DE, 20","4,283483014","17882,22351","35344,22565",33434.0
4,13,12,"440095,53","4475560,73",41813,1,CENTRO,01-05,UNIVERSIDAD,"SAN BERNARDO, CALLE, DE",BiciMAD,24,-3.706220,40.428527,"SAN BERNARDO, CALLE, DE, 87","4,283483014","17882,22351","35344,22565",33434.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,257,249,"441914,27","4481195,42",44194,5,CHAMARTiN,05-06,CASTILLA,"CASTELLANA, PASEO, DE LA",BiciMAD,24,-3.685296,40.479416,"CASTELLANA, PASEO, DE LA, frente al 298","3,561266636","24157,53124","61310,17909",17316.0
262,258,250,"441812,82","4476911,24",44194,5,CHAMARTiN,05-01,EL VISO,"SERRANO, CALLE, DE",BiciMAD,24,-3.686100,40.440815,"SERRANO, CALLE, DE, 113 b","2,196709456","29874,34699","80370,38179",17486.0
263,259,251,"443447,95","4477272,58",44194,5,CHAMARTiN,05-02,PROSPERIDAD,"SANTA HORTENSIA, CALLE, DE",BiciMAD,24,-3.666853,40.444183,"SANTA HORTENSIA, CALLE, DE, 31","4,387920108","20239,33156","46804,54236",37041.0
264,263,255,"441910,45","4479454,04",44194,5,CHAMARTiN,05-05,NUEVA ESPA�A,"GENERAL LOPEZ POZAS, CALLE, DEL",BiciMAD,24,-3.685182,40.463729,"GENERAL LOPEZ POZAS, CALLE, DEL, 2","2,704118379","29308,26982","78097,74713",25105.0


In [12]:
df_EstacionDemog.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 266 entries, 0 to 265
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ESTACION             266 non-null    int64  
 1   Nro_Estacion         266 non-null    object 
 2   Gis_X                266 non-null    object 
 3   Gis_Y                266 non-null    object 
 4   Fec_Alta             266 non-null    object 
 5   Distrito             266 non-null    int64  
 6   Distrito_Nombre      266 non-null    object 
 7   Barrio               266 non-null    object 
 8   Barrio_Nombre        266 non-null    object 
 9   Calle                266 non-null    object 
 10  Tipo_Reserva         266 non-null    object 
 11  Plazas               266 non-null    int64  
 12  Longitud             266 non-null    float64
 13  Latitud              266 non-null    float64
 14  Direccion            266 non-null    object 
 15  Tasa_Paro            264 non-null    obj

### Estacion y movimientos consolidados totales por MES

El objetivo de esta concatenación es agregarle al dataframe de Estaciones los siguientes valores representativo de la demanda:

    - DEMANDA_RATIO = ( DEMANDA_TOTAL / MESES ): valor que permita comparar la demanda general entre estaciones, independiente de si tienen 24 meses de funcionamiento o 12 meses de entrada en vigencia.
    
    - DMNDA_yyyymm = Suma de demanda mensual histórica de viajes realizado desde cada estación por cada MES desde Marzo/2017 hasta Diciembre/2020. 




In [13]:
# Creación de DF inicial con la lista de estaciones

estaciones = data_Tracks.ESTACION.unique()
estaciones = pd.DataFrame(estaciones)
estaciones = estaciones.rename(columns={0:'ESTACION'})
estaciones

,ESTACION
0,1
1,2
2,3
3,4
4,5
...,...
167,139
168,87
169,105
170,146


In [14]:
# Cálculo de meses consolidados totales para cada una de las estaciones

data_Tracks['ANIOMES'] = data_Tracks['ANIO'].astype(str)+("0"+data_Tracks['MES'].astype(str)).str[-2:]

estacion_meses= data_Tracks.groupby('ESTACION').ANIOMES.nunique()
estacion_meses

ESTACION
1      36
2      32
3      36
4      36
5      36
       ..
171    36
172    35
173    35
174    36
175    34
Name: ANIOMES, Length: 172, dtype: int64

In [15]:
# Demanda TOTAL por estacion

df_DMNDA_TOTAL = data_Tracks.groupby('ESTACION').agg(DEMANDA=('DEMANDA', 'sum'))
estacion = pd.merge(df_DMNDA_TOTAL, estacion_meses, how='left', left_on='ESTACION', right_on='ESTACION')
estacion['DEMANDA_RATIO'] = estacion['DEMANDA']/estacion['ANIOMES']
estaciones_tracks = pd.merge(estaciones, estacion['DEMANDA_RATIO'], how='left', left_on='ESTACION', right_on='ESTACION')

estaciones_tracks

,ESTACION,DEMANDA_RATIO
0,1,2505.333333
1,2,1313.468750
2,3,1931.694444
3,4,1342.166667
4,5,1235.027778
...,...,...
167,139,1995.857143
168,87,1119.285714
169,105,929.485714
170,146,824.400000


#### Creación de variables de demanda mensual por estación: df_DMNDA_yyyymm

In [16]:
df_DMNDA_201703 = data_Tracks[(data_Tracks['ANIO']==2017) & (data_Tracks['MES']==3)]
df_DMNDA_201703 = df_DMNDA_201703.groupby('ESTACION').agg(DMNDA_201703=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201703, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201704 = data_Tracks[(data_Tracks['ANIO']==2017) & (data_Tracks['MES']==4)]
df_DMNDA_201704 = df_DMNDA_201704.groupby('ESTACION').agg(DMNDA_201704=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201704, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201705 = data_Tracks[(data_Tracks['ANIO']==2017) & (data_Tracks['MES']==5)]
df_DMNDA_201705 = df_DMNDA_201705.groupby('ESTACION').agg(DMNDA_201705=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201705, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201706 = data_Tracks[(data_Tracks['ANIO']==2017) & (data_Tracks['MES']==6)]
df_DMNDA_201706 = df_DMNDA_201706.groupby('ESTACION').agg(DMNDA_201706=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201706, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201707 = data_Tracks[(data_Tracks['ANIO']==2017) & (data_Tracks['MES']==7)]
df_DMNDA_201707 = df_DMNDA_201707.groupby('ESTACION').agg(DMNDA_201707=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201707, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201708 = data_Tracks[(data_Tracks['ANIO']==2017) & (data_Tracks['MES']==8)]
df_DMNDA_201708 = df_DMNDA_201708.groupby('ESTACION').agg(DMNDA_201708=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201708, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201709 = data_Tracks[(data_Tracks['ANIO']==2017) & (data_Tracks['MES']==9)]
df_DMNDA_201709 = df_DMNDA_201709.groupby('ESTACION').agg(DMNDA_201709=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201709, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201710 = data_Tracks[(data_Tracks['ANIO']==2017) & (data_Tracks['MES']==10)]
df_DMNDA_201710 = df_DMNDA_201710.groupby('ESTACION').agg(DMNDA_201710=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201710, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201711 = data_Tracks[(data_Tracks['ANIO']==2017) & (data_Tracks['MES']==11)]
df_DMNDA_201711 = df_DMNDA_201711.groupby('ESTACION').agg(DMNDA_201711=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201711, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201712 = data_Tracks[(data_Tracks['ANIO']==2017) & (data_Tracks['MES']==12)]
df_DMNDA_201712 = df_DMNDA_201712.groupby('ESTACION').agg(DMNDA_201712=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201712, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201801 = data_Tracks[(data_Tracks['ANIO']==2018) & (data_Tracks['MES']==1)]
df_DMNDA_201801 = df_DMNDA_201801.groupby('ESTACION').agg(DMNDA_201801=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201801, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201802 = data_Tracks[(data_Tracks['ANIO']==2018) & (data_Tracks['MES']==2)]
df_DMNDA_201802 = df_DMNDA_201802.groupby('ESTACION').agg(DMNDA_201802=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201802, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201803 = data_Tracks[(data_Tracks['ANIO']==2018) & (data_Tracks['MES']==3)]
df_DMNDA_201803 = df_DMNDA_201803.groupby('ESTACION').agg(DMNDA_201803=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201803, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201804 = data_Tracks[(data_Tracks['ANIO']==2018) & (data_Tracks['MES']==4)]
df_DMNDA_201804 = df_DMNDA_201804.groupby('ESTACION').agg(DMNDA_201804=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201804, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201805 = data_Tracks[(data_Tracks['ANIO']==2018) & (data_Tracks['MES']==5)]
df_DMNDA_201805 = df_DMNDA_201805.groupby('ESTACION').agg(DMNDA_201805=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201805, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201806 = data_Tracks[(data_Tracks['ANIO']==2018) & (data_Tracks['MES']==6)]
df_DMNDA_201806 = df_DMNDA_201806.groupby('ESTACION').agg(DMNDA_201806=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201806, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201807 = data_Tracks[(data_Tracks['ANIO']==2018) & (data_Tracks['MES']==7)]
df_DMNDA_201807 = df_DMNDA_201807.groupby('ESTACION').agg(DMNDA_201807=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201807, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201808 = data_Tracks[(data_Tracks['ANIO']==2018) & (data_Tracks['MES']==8)]
df_DMNDA_201808 = df_DMNDA_201808.groupby('ESTACION').agg(DMNDA_201808=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201808, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201809 = data_Tracks[(data_Tracks['ANIO']==2018) & (data_Tracks['MES']==9)]
df_DMNDA_201809 = df_DMNDA_201809.groupby('ESTACION').agg(DMNDA_201809=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201809, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201810 = data_Tracks[(data_Tracks['ANIO']==2018) & (data_Tracks['MES']==10)]
df_DMNDA_201810 = df_DMNDA_201810.groupby('ESTACION').agg(DMNDA_201810=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201810, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201811 = data_Tracks[(data_Tracks['ANIO']==2018) & (data_Tracks['MES']==11)]
df_DMNDA_201811 = df_DMNDA_201811.groupby('ESTACION').agg(DMNDA_201811=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201811, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201812 = data_Tracks[(data_Tracks['ANIO']==2018) & (data_Tracks['MES']==12)]
df_DMNDA_201812 = df_DMNDA_201812.groupby('ESTACION').agg(DMNDA_201812=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201812, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201901 = data_Tracks[(data_Tracks['ANIO']==2019) & (data_Tracks['MES']==1)]
df_DMNDA_201901 = df_DMNDA_201901.groupby('ESTACION').agg(DMNDA_201901=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201901, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201902 = data_Tracks[(data_Tracks['ANIO']==2019) & (data_Tracks['MES']==2)]
df_DMNDA_201902 = df_DMNDA_201902.groupby('ESTACION').agg(DMNDA_201902=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201902, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201903 = data_Tracks[(data_Tracks['ANIO']==2019) & (data_Tracks['MES']==3)]
df_DMNDA_201903 = df_DMNDA_201903.groupby('ESTACION').agg(DMNDA_201903=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201903, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201904 = data_Tracks[(data_Tracks['ANIO']==2019) & (data_Tracks['MES']==4)]
df_DMNDA_201904 = df_DMNDA_201904.groupby('ESTACION').agg(DMNDA_201904=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201904, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201905 = data_Tracks[(data_Tracks['ANIO']==2019) & (data_Tracks['MES']==5)]
df_DMNDA_201905 = df_DMNDA_201905.groupby('ESTACION').agg(DMNDA_201905=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201905, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201906 = data_Tracks[(data_Tracks['ANIO']==2019) & (data_Tracks['MES']==6)]
df_DMNDA_201906 = df_DMNDA_201906.groupby('ESTACION').agg(DMNDA_201906=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201906, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201907 = data_Tracks[(data_Tracks['ANIO']==2019) & (data_Tracks['MES']==7)]
df_DMNDA_201907 = df_DMNDA_201907.groupby('ESTACION').agg(DMNDA_201907=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201907, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201908 = data_Tracks[(data_Tracks['ANIO']==2019) & (data_Tracks['MES']==8)]
df_DMNDA_201908 = df_DMNDA_201908.groupby('ESTACION').agg(DMNDA_201908=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201908, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201909 = data_Tracks[(data_Tracks['ANIO']==2019) & (data_Tracks['MES']==9)]
df_DMNDA_201909 = df_DMNDA_201909.groupby('ESTACION').agg(DMNDA_201909=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201909, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201910 = data_Tracks[(data_Tracks['ANIO']==2019) & (data_Tracks['MES']==10)]
df_DMNDA_201910 = df_DMNDA_201910.groupby('ESTACION').agg(DMNDA_201910=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201910, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201911 = data_Tracks[(data_Tracks['ANIO']==2019) & (data_Tracks['MES']==11)]
df_DMNDA_201911 = df_DMNDA_201911.groupby('ESTACION').agg(DMNDA_201911=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201911, how='left', left_on='ESTACION', right_on='ESTACION')

df_DMNDA_201912 = data_Tracks[(data_Tracks['ANIO']==2019) & (data_Tracks['MES']==12)]
df_DMNDA_201912 = df_DMNDA_201912.groupby('ESTACION').agg(DMNDA_201912=('DEMANDA', 'sum'))
estaciones_tracks = pd.merge(estaciones_tracks, df_DMNDA_201912, how='left', left_on='ESTACION', right_on='ESTACION')

#### El dataframe final contiene las variables DEMANDA_RATIO y DMNDA_yyyymm para cada una de las estaciones

In [17]:
estaciones_tracks.fillna(0, inplace=True)
estaciones_tracks

,ESTACION,DEMANDA_RATIO,DMNDA_201703,DMNDA_201704,DMNDA_201705,DMNDA_201706,DMNDA_201707,DMNDA_201708,DMNDA_201709,DMNDA_201710,...,DMNDA_201903,DMNDA_201904,DMNDA_201905,DMNDA_201906,DMNDA_201907,DMNDA_201908,DMNDA_201909,DMNDA_201910,DMNDA_201911,DMNDA_201912
0,1,2505.333333,7.0,2212.0,2942,2946,3229,2929,3685,3606,...,2618.0,2581.0,2948.0,2850.0,2824.0,2304.0,3133.0,3524.0,1306.0,29.0
1,2,1313.468750,3.0,1174.0,1579,1536,1597,1491,1924,2033,...,1266.0,1125.0,1819.0,1678.0,1499.0,1289.0,825.0,0.0,0.0,0.0
2,3,1931.694444,1.0,1338.0,1776,2398,2108,1862,2270,2193,...,1974.0,2052.0,2929.0,3243.0,2579.0,2016.0,2329.0,2246.0,1700.0,1288.0
3,4,1342.166667,1.0,1131.0,1154,1801,1522,1265,1873,1843,...,1142.0,1259.0,1876.0,2098.0,1282.0,1219.0,1784.0,1767.0,1260.0,663.0
4,5,1235.027778,2.0,1052.0,1318,1573,1453,1201,1597,1594,...,1222.0,1297.0,1840.0,1760.0,1433.0,1095.0,1451.0,1403.0,1116.0,739.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,139,1995.857143,0.0,1488.0,2031,2384,2312,2004,2643,2599,...,1887.0,1942.0,2768.0,2975.0,2392.0,1927.0,2142.0,1989.0,1412.0,1118.0
168,87,1119.285714,0.0,863.0,1127,1156,1059,846,1327,1438,...,1131.0,971.0,1464.0,1471.0,1305.0,874.0,1289.0,1445.0,1085.0,832.0
169,105,929.485714,0.0,785.0,1005,1115,978,716,1125,1150,...,951.0,836.0,1188.0,1397.0,1193.0,825.0,1267.0,1339.0,957.0,652.0
170,146,824.400000,0.0,845.0,1046,1162,856,647,1190,1230,...,721.0,645.0,863.0,976.0,791.0,650.0,1025.0,1077.0,698.0,446.0


In [18]:
data_Tracks['ANIOMES'] = data_Tracks['ANIO'].astype(str)+("0"+data_Tracks['MES'].astype(str)).str[-2:]
data_Tracks

,ESTACION,ANIO,MES,DIA,HORA,FECHA,DIA_SEMANA,AM_PM,TEMPORADA,TEMPORADA_NUM,...,TEMPERATURA,VIENTO,PRESION,HUMEDAD,PRECIPITACION_1h,PRECIPITACION_3h,DESC_TIEMPO,DESC_TIEMPO_detalle,DEMANDA,ANIOMES
0,1,2017,3,31,23,2017-03-31 23:00:00,6,PM,INVIERNO,1,...,12.58,7.72,1020.0,44.0,0.0,0.0,Clouds,few clouds,7,201703
1,2,2017,3,31,23,2017-03-31 23:00:00,6,PM,INVIERNO,1,...,12.58,7.72,1020.0,44.0,0.0,0.0,Clouds,few clouds,3,201703
2,3,2017,3,31,23,2017-03-31 23:00:00,6,PM,INVIERNO,1,...,12.58,7.72,1020.0,44.0,0.0,0.0,Clouds,few clouds,1,201703
3,4,2017,3,31,23,2017-03-31 23:00:00,6,PM,INVIERNO,1,...,12.58,7.72,1020.0,44.0,0.0,0.0,Clouds,few clouds,1,201703
4,5,2017,3,31,23,2017-03-31 23:00:00,6,PM,INVIERNO,1,...,12.58,7.72,1020.0,44.0,0.0,0.0,Clouds,few clouds,2,201703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2912133,168,2020,2,29,23,2020-02-29 23:00:00,7,PM,INVIERNO,1,...,9.60,6.20,1015.0,69.0,0.0,0.0,Clouds,few clouds,2,202002
2912134,169,2020,2,29,23,2020-02-29 23:00:00,7,PM,INVIERNO,1,...,9.60,6.20,1015.0,69.0,0.0,0.0,Clouds,few clouds,8,202002
2912135,171,2020,2,29,23,2020-02-29 23:00:00,7,PM,INVIERNO,1,...,9.60,6.20,1015.0,69.0,0.0,0.0,Clouds,few clouds,2,202002
2912136,172,2020,2,29,23,2020-02-29 23:00:00,7,PM,INVIERNO,1,...,9.60,6.20,1015.0,69.0,0.0,0.0,Clouds,few clouds,1,202002


In [19]:
estaciones_tracks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172 entries, 0 to 171
Data columns (total 36 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ESTACION       172 non-null    int64  
 1   DEMANDA_RATIO  172 non-null    float64
 2   DMNDA_201703   172 non-null    float64
 3   DMNDA_201704   172 non-null    float64
 4   DMNDA_201705   172 non-null    int64  
 5   DMNDA_201706   172 non-null    int64  
 6   DMNDA_201707   172 non-null    int64  
 7   DMNDA_201708   172 non-null    int64  
 8   DMNDA_201709   172 non-null    int64  
 9   DMNDA_201710   172 non-null    int64  
 10  DMNDA_201711   172 non-null    int64  
 11  DMNDA_201712   172 non-null    float64
 12  DMNDA_201801   172 non-null    float64
 13  DMNDA_201802   172 non-null    int64  
 14  DMNDA_201803   172 non-null    int64  
 15  DMNDA_201804   172 non-null    int64  
 16  DMNDA_201805   172 non-null    float64
 17  DMNDA_201806   172 non-null    float64
 18  DMNDA_2018

### Concatenación de Estaciones con Datos Demográficos por Barrio

In [22]:
df_FinalCluster = pd.merge(estaciones_tracks, df_EstacionDemog, how='left', left_on='ESTACION', right_on='ESTACION')
df_FinalCluster = df_FinalCluster.drop(columns=['Nro_Estacion', 'Fec_Alta', 'Distrito_Nombre', 'Barrio_Nombre', 
                                                'Calle', 'Direccion', 'Tipo_Reserva'])
df_FinalCluster


,ESTACION,DEMANDA_RATIO,DMNDA_201703,DMNDA_201704,DMNDA_201705,DMNDA_201706,DMNDA_201707,DMNDA_201708,DMNDA_201709,DMNDA_201710,...,Gis_Y,Distrito,Barrio,Plazas,Longitud,Latitud,Tasa_Paro,Renta_Media_Persona,Renta_Media_Hogar,Poblacion
0,1,2505.333333,7.0,2212.0,2942,2946,3229,2929,3685,3606,...,"4474290,65",1,01-06,30,-3.701998,40.417111,"4,309681391","17550,00279","35056,88536",7665.0
1,2,1313.468750,3.0,1174.0,1579,1536,1597,1491,1924,2033,...,"4474301,74",1,01-06,30,-3.701564,40.417213,"4,309681391","17550,00279","35056,88536",7665.0
2,3,1931.694444,1.0,1338.0,1776,2398,2108,1862,2270,2193,...,"4474678,23",1,01-05,24,-3.705674,40.420580,"4,283483014","17882,22351","35344,22565",33434.0
3,4,1342.166667,1.0,1131.0,1154,1801,1522,1265,1873,1843,...,"4475760,68",7,07-02,18,-3.707212,40.430322,"3,985756232","21397,21963","46882,35448",24770.0
4,5,1235.027778,2.0,1052.0,1318,1573,1453,1201,1597,1594,...,"4475565,36",1,01-05,24,-3.702674,40.428590,"4,283483014","17882,22351","35344,22565",33434.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,139,1995.857143,0.0,1488.0,2031,2384,2312,2004,2643,2599,...,"4478749,38",6,06-02,24,-3.700864,40.457286,"3,941354032","19995,56514","46339,28875",35395.0
168,87,1119.285714,0.0,863.0,1127,1156,1059,846,1327,1438,...,"4473311,29",3,03-03,24,-3.669683,40.408479,"3,593663803","23696,69349","63495,10119",23504.0
169,105,929.485714,0.0,785.0,1005,1115,978,716,1125,1150,...,"4475123,85",4,04-02,24,-3.673878,40.424784,"3,198151951","24044,83181","54456,35478",30050.0
170,146,824.400000,0.0,845.0,1046,1162,856,647,1190,1230,...,"4477914,69",5,05-01,24,-3.687851,40.449845,"2,196709456","29874,34699","80370,38179",17486.0


In [23]:
df_FinalCluster.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172 entries, 0 to 171
Data columns (total 47 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ESTACION             172 non-null    int64  
 1   DEMANDA_RATIO        172 non-null    float64
 2   DMNDA_201703         172 non-null    float64
 3   DMNDA_201704         172 non-null    float64
 4   DMNDA_201705         172 non-null    int64  
 5   DMNDA_201706         172 non-null    int64  
 6   DMNDA_201707         172 non-null    int64  
 7   DMNDA_201708         172 non-null    int64  
 8   DMNDA_201709         172 non-null    int64  
 9   DMNDA_201710         172 non-null    int64  
 10  DMNDA_201711         172 non-null    int64  
 11  DMNDA_201712         172 non-null    float64
 12  DMNDA_201801         172 non-null    float64
 13  DMNDA_201802         172 non-null    int64  
 14  DMNDA_201803         172 non-null    int64  
 15  DMNDA_201804         172 non-null    int

In [24]:
# Cambios de tipos de datos para variables numéricas

df_FinalCluster['Gis_X'] = df_FinalCluster.Gis_X.astype(str)
df_FinalCluster['Gis_X'] = [x.replace(',', '.') for x in df_FinalCluster['Gis_X']]
df_FinalCluster['Gis_X'] = df_FinalCluster.Gis_X.astype(float)

df_FinalCluster['Gis_Y'] = df_FinalCluster.Gis_Y.astype(str)
df_FinalCluster['Gis_Y'] = [x.replace(',', '.') for x in df_FinalCluster['Gis_Y']]
df_FinalCluster['Gis_Y'] = df_FinalCluster.Gis_Y.astype(float)

#df_FinalCluster['Finca'] = df_FinalCluster.Finca.astype('string')
df_FinalCluster['Barrio'] = df_FinalCluster.Barrio.astype('string')

df_FinalCluster['Tasa_Paro'] = df_FinalCluster.Tasa_Paro.astype(str)
df_FinalCluster['Tasa_Paro'] = [x.replace(',', '.') for x in df_FinalCluster['Tasa_Paro']]
df_FinalCluster['Tasa_Paro'] = df_FinalCluster.Tasa_Paro.astype(float)

df_FinalCluster['Renta_Media_Persona'] = df_FinalCluster.Renta_Media_Persona.astype(str)
df_FinalCluster['Renta_Media_Persona'] = [x.replace(',', '.') for x in df_FinalCluster['Renta_Media_Persona']]
df_FinalCluster['Renta_Media_Persona'] = df_FinalCluster.Renta_Media_Persona.astype(float)

df_FinalCluster['Renta_Media_Hogar'] = df_FinalCluster.Renta_Media_Hogar.astype(str)
df_FinalCluster['Renta_Media_Hogar'] = [x.replace(',', '.') for x in df_FinalCluster['Renta_Media_Hogar']]
df_FinalCluster['Renta_Media_Hogar'] = df_FinalCluster.Renta_Media_Hogar.astype(float)

df_FinalCluster['Gis_X'] = df_FinalCluster.Gis_X.astype(str)
df_FinalCluster['Gis_X'] = [x.replace(',', '.') for x in df_FinalCluster['Gis_X']]
df_FinalCluster['Gis_X'] = df_FinalCluster.Gis_X.astype(float)

df_FinalCluster['Gis_Y'] = df_FinalCluster.Gis_Y.astype(str)
df_FinalCluster['Gis_Y'] = [x.replace(',', '.') for x in df_FinalCluster['Gis_Y']]
df_FinalCluster['Gis_Y'] = df_FinalCluster.Gis_Y.astype(float)


df_FinalCluster.info()
df_FinalCluster


<class 'pandas.core.frame.DataFrame'>
Int64Index: 172 entries, 0 to 171
Data columns (total 47 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ESTACION             172 non-null    int64  
 1   DEMANDA_RATIO        172 non-null    float64
 2   DMNDA_201703         172 non-null    float64
 3   DMNDA_201704         172 non-null    float64
 4   DMNDA_201705         172 non-null    int64  
 5   DMNDA_201706         172 non-null    int64  
 6   DMNDA_201707         172 non-null    int64  
 7   DMNDA_201708         172 non-null    int64  
 8   DMNDA_201709         172 non-null    int64  
 9   DMNDA_201710         172 non-null    int64  
 10  DMNDA_201711         172 non-null    int64  
 11  DMNDA_201712         172 non-null    float64
 12  DMNDA_201801         172 non-null    float64
 13  DMNDA_201802         172 non-null    int64  
 14  DMNDA_201803         172 non-null    int64  
 15  DMNDA_201804         172 non-null    int

,ESTACION,DEMANDA_RATIO,DMNDA_201703,DMNDA_201704,DMNDA_201705,DMNDA_201706,DMNDA_201707,DMNDA_201708,DMNDA_201709,DMNDA_201710,...,Gis_Y,Distrito,Barrio,Plazas,Longitud,Latitud,Tasa_Paro,Renta_Media_Persona,Renta_Media_Hogar,Poblacion
0,1,2505.333333,7.0,2212.0,2942,2946,3229,2929,3685,3606,...,4474290.65,1,01-06,30,-3.701998,40.417111,4.309681,17550.00279,35056.88536,7665.0
1,2,1313.468750,3.0,1174.0,1579,1536,1597,1491,1924,2033,...,4474301.74,1,01-06,30,-3.701564,40.417213,4.309681,17550.00279,35056.88536,7665.0
2,3,1931.694444,1.0,1338.0,1776,2398,2108,1862,2270,2193,...,4474678.23,1,01-05,24,-3.705674,40.420580,4.283483,17882.22351,35344.22565,33434.0
3,4,1342.166667,1.0,1131.0,1154,1801,1522,1265,1873,1843,...,4475760.68,7,07-02,18,-3.707212,40.430322,3.985756,21397.21963,46882.35448,24770.0
4,5,1235.027778,2.0,1052.0,1318,1573,1453,1201,1597,1594,...,4475565.36,1,01-05,24,-3.702674,40.428590,4.283483,17882.22351,35344.22565,33434.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,139,1995.857143,0.0,1488.0,2031,2384,2312,2004,2643,2599,...,4478749.38,6,06-02,24,-3.700864,40.457286,3.941354,19995.56514,46339.28875,35395.0
168,87,1119.285714,0.0,863.0,1127,1156,1059,846,1327,1438,...,4473311.29,3,03-03,24,-3.669683,40.408479,3.593664,23696.69349,63495.10119,23504.0
169,105,929.485714,0.0,785.0,1005,1115,978,716,1125,1150,...,4475123.85,4,04-02,24,-3.673878,40.424784,3.198152,24044.83181,54456.35478,30050.0
170,146,824.400000,0.0,845.0,1046,1162,856,647,1190,1230,...,4477914.69,5,05-01,24,-3.687851,40.449845,2.196709,29874.34699,80370.38179,17486.0


## 3. Exportar DataFrame a CSV

In [39]:
df_FinalCluster.to_csv('../../data/df_FinalCluster_2017_2019.csv', index=False)